In [1]:
import numpy as np
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect
from flask import Flask, jsonify


# Database Setup
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)

# Flask Setup
app = Flask(__name__)


@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Welcome to the Homepage of Hawaii Precipitation and Temperature data<br/>"
        f"<br/>"
        f"Available routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start_date<br/>"
        f"/api/v1.0/start_date/end_date/"
    )

#Convert the query results to a dictionary using date as the key and prcp as the value.
#Return the JSON representation of your dictionary.


@app.route("/api/v1.0/precipitation/")
def precipitation():
    
    session = Session(engine)
    last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
    last_date = dt.datetime.strptime(last_date, "%Y-%m-%d")
    first_date = last_date - dt.timedelta(days=365)
    last_year_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= first_date).all()
    
    #all_prcp = []
    #for date, prcp in last_year_data:
        #prcp_dict = {}
        #prcp_dict["date"] = date
        #prcp_dict["prcp"] = prcp
        #all_prcp.append(prcp_dict)
  
    
    session.close()
    return jsonify(last_year_data)
    #return jsonify(all_prcp)

@app.route("/api/v1.0/stations/")
def station():
    session = Session(engine)
    stations = session.query(Station.station).all()
    session.close()
    return jsonify(stations)
    

@app.route("/api/v1.0/tobs/")
def tobs():
    session = Session(engine)
    stations = session.query(Measurement.station,func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
    most_active_station = stations[0][0]
    station_data = session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == most_active_station).all()
    station_data = list(np.ravel(station_data))
    session.close()
    return jsonify(station_data)
    

@app.route('/api/v1.0/<start_date>/')
def calc_temps_start(start_date):
    session = Session(engine)
    results = session.query(func.min(Measurement.tobs),func.avg(Measurement.tobs),func.max(Measurement.tobs)).filter(Measurement.date > start_date).all()
    temps = list(np.ravel(results))
    session.close()
    return jsonify(temps)


# Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start-end range.
@app.route('/api/v1.0/<start_date>/<end_date>/')
def calc_temps_start_end(start_date, end_date):
    session = Session(engine)
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
    temps = list(np.ravel(results))
    session.close()
    return jsonify(temps)



# 4. Define main behavior
if __name__ == "__main__":
    app.run(debug=False)



 * Serving Flask app &quot;__main__&quot; (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Oct/2020 16:14:03] &quot;GET / HTTP/1.1&quot; 200 -
127.0.0.1 - - [06/Oct/2020 16:14:12] &quot;GET /api/v1.0/precipitation/ HTTP/1.1&quot; 200 -
127.0.0.1 - - [06/Oct/2020 16:17:05] &quot;GET / HTTP/1.1&quot; 200 -
127.0.0.1 - - [06/Oct/2020 16:17:12] &quot;GET /api/v1.0/precipitation/ HTTP/1.1&quot; 200 -
